In [2]:
import subprocess
import sqlite3
import json
from datetime import datetime
import import_ipynb
import utils 

def fetch_tee_times():
    """Fetches tee time data from the API using curl."""
    course_date = datetime.now().strftime('%Y-%m-%d')
    curl_command = [
        "curl",
        "--ssl-no-revoke",
        f"https://greenvalleyranch.cps.golf/onlineres/onlineapi/api/v1/onlinereservation/TeeSheetNotes?courseIds=1&courseDate={course_date}",
        "-H", "accept: application/json, text/plain, */*",
        "-H", "accept-language: en-GB,en-US;q=0.9,en;q=0.8",
        "-H", "cache-control: no-cache, no-store, must-revalidate",
        "-H", "client-id: onlineresweb",
        "-H", "cookie: V4COOKIE=1736954495.412.720.99506|beafa6f5988e5b196bba96a1f5b9227f",
        "-H", "expires: Sat, 01 Jan 2000 00:00:00 GMT",
        "-H", "if-modified-since: 0",
        "-H", "pragma: no-cache",
        "-H", "priority: u=1, i",
        "-H", "referer: https://greenvalleyranch.cps.golf/onlineresweb/search-teetime?TeeOffTimeMin=0&TeeOffTimeMax=23",
        "-H", "sec-ch-ua: \"Google Chrome\";v=\"131\", \"Chromium\";v=\"131\", \"Not_A Brand\";v=\"24\"",
        "-H", "sec-ch-ua-mobile: ?0",
        "-H", "sec-ch-ua-platform: \"Windows\"",
        "-H", "sec-fetch-dest: empty",
        "-H", "sec-fetch-mode: cors",
        "-H", "sec-fetch-site: same-origin",
        "-H", "user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "-H", "x-apikey: 8ea2914e-cac2-48a7-a3e5-e0f41350bf3a",
        "-H", "x-componentid: 1",
        "-H", "x-ismobile: false",
        "-H", "x-moduleid: 7",
        "-H", "x-productid: 1",
        "-H", "x-requestid: fd5193a9-8e6e-3d5b-6121-869eb3eeb286",
        "-H", "x-siteid: 2",
        "-H", "x-terminalid: 3",
        "-H", "x-timezone-offset: -330",
        "-H", "x-timezoneid: Asia/Calcutta",
        "-H", "x-websiteid: e6b92812-d6c4-4f86-7eea-08d9fadf154d"
    ]
    result = subprocess.run(curl_command, capture_output=True, text=True)
    return json.loads(result.stdout) if result.stdout else []

def parse_tee_time_data(data):
    """Parses raw JSON tee time data into a structured format."""
    parsed_data = []
    for item in data:
        parsed_data.append((
            item.get('courseDate', ''),
            item.get('courseId', 0),
            item.get('courseName', ''),
            item.get('startTime', ''),
            int(item.get('playersDisplay', '0').split()[0]) if item.get('playersDisplay') else 0,
            item.get('holes', 0),
            float(item.get('shItemPrices', [{}])[0].get('price', 0))
        ))
    return parsed_data

def get_greenvalleyrach_tee_time():
    """Main function to fetch, parse, and update tee time data."""
    raw_data = fetch_tee_times()
    if raw_data:
        parsed_data = parse_tee_time_data(raw_data)
        utils.update_database(parsed_data, table_name="TeeTimes")
        print("Green Valley Ranch data updated in GolfteeTimes_database.db")
    else:
        print("No data fetched.")
    
    return parsed_data  # Return the data for use in utils.main()

if __name__ == "__main__":
    utils.initialize_database()
    get_greenvalleyrach_tee_time()


Green Valley Ranch data updated in GolfteeTimes_database.db
